In [1]:
pip install pandas

  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.8 MB 1.3 MB/s eta 0:00:09
    --------------------------------------- 0.2/10.8 MB 3.7 MB/s eta 0:00:03
   --- ------------------------------------ 0.8/10.8 MB 7.7 MB/s eta 0:00:02
   ------ --------------------------------- 1.8/10.8 MB 11.5 MB/s eta 0:00:01
   ----------- ---------------------------- 3.0/10.8 MB 14.7 MB/s eta 0:00:01
   ------------- -------------------------- 3.7/10.8 MB 13.8 MB/s eta 0:00:01
   -------------- ------------------------- 4.0/10.8 MB 13.3 MB/s eta 0:00:01
   ---------------- ----------------------- 4.4/10.8 MB 12.8 MB/s eta 0:00:01
   ------------------ --------------------- 5.1/10.8 MB 12.9 MB/s eta 0:00:01
   -------------------- ------------------- 5.5/10.8 MB 12.4 MB/s eta 0:00:01
   ------------------------ --------------- 6.7/10.8 MB 13.8 MB/s eta 0:00:01
   ----

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-gpu 2.4.1 requires numpy~=1.19.2, but you have numpy 1.24.4 which is incompatible.


In [3]:
pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached setuptools-71.1.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached tensorboard-2.17.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached keras-3.4.1-py3-none-any.whl.metadata (5.8 kB)
  Using cached wheel-0.43.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached namex-0.0.8-py3-none-any.whl.metadata (246 bytes)
     ---------------------------------------- 0.0/48.7 kB ? eta -:--:--
     ---------------------------------------- 

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: C:\Python312\python.exe -m pip install --upgrade pip


In [4]:
pip install python-dotenv

  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
import pandas
from dotenv import load_dotenv
import os
import requests
import sklearn
from sklearn.model_selection import train_test_split
import tensorflow

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
tensorflow.config.list_physical_devices('GPU')

[]

In [4]:
farfetchData=pandas.read_csv("D:/farfetchdataset/current_farfetch_listings.csv")

In [5]:
farfetchData

,Unnamed: 0,availableSizes,brand.id,brand.name,gender,hasSimilarProducts,id,images.cutOut,images.model,isCustomizable,...,priceInfo.currencyCode,priceInfo.discountLabel,priceInfo.finalPrice,priceInfo.formattedFinalPrice,priceInfo.formattedInitialPrice,priceInfo.initialPrice,priceInfo.installmentsLabel,priceInfo.isOnSale,shortDescription,stockTotal
0,0,NaN,8360,Miu Miu,women,True,13415450,https://cdn-images.farfetch-contents.com/13/41...,https://cdn-images.farfetch-contents.com/13/41...,False,...,SGD,NaN,1080,"$1,080","$1,080",1080,NaN,False,logo print strap sandals,130
1,1,"[{'scaleId': 0, 'size': 'XXS'}, {'scaleId': 0,...",2049368,Self-Portrait,women,True,13784566,https://cdn-images.farfetch-contents.com/13/78...,https://cdn-images.farfetch-contents.com/13/78...,False,...,SGD,20% Off,392,$392,$490,490,NaN,True,embroidered midi dress,14
2,2,NaN,15514,Fendi,women,True,13377441,https://cdn-images.farfetch-contents.com/13/37...,https://cdn-images.farfetch-contents.com/13/37...,False,...,SGD,NaN,1245,"$1,245","$1,245",1245,NaN,False,FendiMania sock style sneakers,68
3,3,"[{'scaleId': 0, 'size': 'One Size'}]",49486,A.P.C.,women,False,13744082,https://cdn-images.farfetch-contents.com/13/74...,https://cdn-images.farfetch-contents.com/13/74...,False,...,SGD,20% Off,308,$308,$385,385,NaN,True,top zip wallet,1
4,4,NaN,34624,Prada,women,True,13668256,https://cdn-images.farfetch-contents.com/13/66...,https://cdn-images.farfetch-contents.com/13/66...,False,...,SGD,NaN,1248,"$1,248","$1,248",1248,NaN,False,slingback 65 pumps,78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188812,172,"[{'scaleId': 0, 'size': 'One Size'}]",59394,Fornasetti,unisex,False,10047577,https://cdn-images.farfetch-contents.com/10/04...,https://cdn-images.farfetch-contents.com/10/04...,False,...,SGD,NaN,207,$207,$207,207,NaN,False,face print plate,6
188813,173,"[{'scaleId': 0, 'size': 'One Size'}]",59394,Fornasetti,unisex,False,10047571,https://cdn-images.farfetch-contents.com/10/04...,https://cdn-images.farfetch-contents.com/10/04...,False,...,SGD,NaN,207,$207,$207,207,NaN,False,boss print plate,2
188814,174,"[{'scaleId': 0, 'size': 'One Size'}]",59394,Fornasetti,unisex,False,10047569,https://cdn-images.farfetch-contents.com/10/04...,https://cdn-images.farfetch-contents.com/10/04...,False,...,SGD,NaN,207,$207,$207,207,NaN,False,face print plate,2
188815,175,"[{'scaleId': 0, 'size': 'One Size'}]",59394,Fornasetti,unisex,False,10047567,https://cdn-images.farfetch-contents.com/10/04...,https://cdn-images.farfetch-contents.com/10/04...,False,...,SGD,NaN,207,$207,$207,207,NaN,False,whisker print plate,10


In [6]:
load_dotenv()

True

In [7]:
CURRENCY_API_KEY=os.getenv("CURRENCY_API_KEY")

In [8]:
def getExchangeRates(currency):
    url='https://v6.exchangerate-api.com/v6/'+CURRENCY_API_KEY+'/latest/'+currency
    response=requests.get(url)
    return response.json()

In [9]:
sgd=getExchangeRates('SGD')

In [10]:
sgd['conversion_rates']['USD']

0.7443

In [11]:
farfetchData['priceInfo.initialPrice']=farfetchData['priceInfo.initialPrice'].map(lambda x: x*sgd['conversion_rates']['USD'])

In [12]:
farfetchData['priceInfo.finalPrice']=farfetchData['priceInfo.finalPrice'].map(lambda x: x*sgd['conversion_rates']['USD'])

In [13]:
farfetchData

,Unnamed: 0,availableSizes,brand.id,brand.name,gender,hasSimilarProducts,id,images.cutOut,images.model,isCustomizable,...,priceInfo.currencyCode,priceInfo.discountLabel,priceInfo.finalPrice,priceInfo.formattedFinalPrice,priceInfo.formattedInitialPrice,priceInfo.initialPrice,priceInfo.installmentsLabel,priceInfo.isOnSale,shortDescription,stockTotal
0,0,NaN,8360,Miu Miu,women,True,13415450,https://cdn-images.farfetch-contents.com/13/41...,https://cdn-images.farfetch-contents.com/13/41...,False,...,SGD,NaN,803.8440,"$1,080","$1,080",803.8440,NaN,False,logo print strap sandals,130
1,1,"[{'scaleId': 0, 'size': 'XXS'}, {'scaleId': 0,...",2049368,Self-Portrait,women,True,13784566,https://cdn-images.farfetch-contents.com/13/78...,https://cdn-images.farfetch-contents.com/13/78...,False,...,SGD,20% Off,291.7656,$392,$490,364.7070,NaN,True,embroidered midi dress,14
2,2,NaN,15514,Fendi,women,True,13377441,https://cdn-images.farfetch-contents.com/13/37...,https://cdn-images.farfetch-contents.com/13/37...,False,...,SGD,NaN,926.6535,"$1,245","$1,245",926.6535,NaN,False,FendiMania sock style sneakers,68
3,3,"[{'scaleId': 0, 'size': 'One Size'}]",49486,A.P.C.,women,False,13744082,https://cdn-images.farfetch-contents.com/13/74...,https://cdn-images.farfetch-contents.com/13/74...,False,...,SGD,20% Off,229.2444,$308,$385,286.5555,NaN,True,top zip wallet,1
4,4,NaN,34624,Prada,women,True,13668256,https://cdn-images.farfetch-contents.com/13/66...,https://cdn-images.farfetch-contents.com/13/66...,False,...,SGD,NaN,928.8864,"$1,248","$1,248",928.8864,NaN,False,slingback 65 pumps,78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188812,172,"[{'scaleId': 0, 'size': 'One Size'}]",59394,Fornasetti,unisex,False,10047577,https://cdn-images.farfetch-contents.com/10/04...,https://cdn-images.farfetch-contents.com/10/04...,False,...,SGD,NaN,154.0701,$207,$207,154.0701,NaN,False,face print plate,6
188813,173,"[{'scaleId': 0, 'size': 'One Size'}]",59394,Fornasetti,unisex,False,10047571,https://cdn-images.farfetch-contents.com/10/04...,https://cdn-images.farfetch-contents.com/10/04...,False,...,SGD,NaN,154.0701,$207,$207,154.0701,NaN,False,boss print plate,2
188814,174,"[{'scaleId': 0, 'size': 'One Size'}]",59394,Fornasetti,unisex,False,10047569,https://cdn-images.farfetch-contents.com/10/04...,https://cdn-images.farfetch-contents.com/10/04...,False,...,SGD,NaN,154.0701,$207,$207,154.0701,NaN,False,face print plate,2
188815,175,"[{'scaleId': 0, 'size': 'One Size'}]",59394,Fornasetti,unisex,False,10047567,https://cdn-images.farfetch-contents.com/10/04...,https://cdn-images.farfetch-contents.com/10/04...,False,...,SGD,NaN,154.0701,$207,$207,154.0701,NaN,False,whisker print plate,10


In [14]:
def roundPrices(price):
    if price<100:
        return round(price,-1)
    else: return round(price,-2)

In [15]:
farfetchData['priceInfo.initialPrice']=farfetchData['priceInfo.initialPrice'].map(roundPrices)

In [16]:
farfetchData

,Unnamed: 0,availableSizes,brand.id,brand.name,gender,hasSimilarProducts,id,images.cutOut,images.model,isCustomizable,...,priceInfo.currencyCode,priceInfo.discountLabel,priceInfo.finalPrice,priceInfo.formattedFinalPrice,priceInfo.formattedInitialPrice,priceInfo.initialPrice,priceInfo.installmentsLabel,priceInfo.isOnSale,shortDescription,stockTotal
0,0,NaN,8360,Miu Miu,women,True,13415450,https://cdn-images.farfetch-contents.com/13/41...,https://cdn-images.farfetch-contents.com/13/41...,False,...,SGD,NaN,803.8440,"$1,080","$1,080",800.0,NaN,False,logo print strap sandals,130
1,1,"[{'scaleId': 0, 'size': 'XXS'}, {'scaleId': 0,...",2049368,Self-Portrait,women,True,13784566,https://cdn-images.farfetch-contents.com/13/78...,https://cdn-images.farfetch-contents.com/13/78...,False,...,SGD,20% Off,291.7656,$392,$490,400.0,NaN,True,embroidered midi dress,14
2,2,NaN,15514,Fendi,women,True,13377441,https://cdn-images.farfetch-contents.com/13/37...,https://cdn-images.farfetch-contents.com/13/37...,False,...,SGD,NaN,926.6535,"$1,245","$1,245",900.0,NaN,False,FendiMania sock style sneakers,68
3,3,"[{'scaleId': 0, 'size': 'One Size'}]",49486,A.P.C.,women,False,13744082,https://cdn-images.farfetch-contents.com/13/74...,https://cdn-images.farfetch-contents.com/13/74...,False,...,SGD,20% Off,229.2444,$308,$385,300.0,NaN,True,top zip wallet,1
4,4,NaN,34624,Prada,women,True,13668256,https://cdn-images.farfetch-contents.com/13/66...,https://cdn-images.farfetch-contents.com/13/66...,False,...,SGD,NaN,928.8864,"$1,248","$1,248",900.0,NaN,False,slingback 65 pumps,78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188812,172,"[{'scaleId': 0, 'size': 'One Size'}]",59394,Fornasetti,unisex,False,10047577,https://cdn-images.farfetch-contents.com/10/04...,https://cdn-images.farfetch-contents.com/10/04...,False,...,SGD,NaN,154.0701,$207,$207,200.0,NaN,False,face print plate,6
188813,173,"[{'scaleId': 0, 'size': 'One Size'}]",59394,Fornasetti,unisex,False,10047571,https://cdn-images.farfetch-contents.com/10/04...,https://cdn-images.farfetch-contents.com/10/04...,False,...,SGD,NaN,154.0701,$207,$207,200.0,NaN,False,boss print plate,2
188814,174,"[{'scaleId': 0, 'size': 'One Size'}]",59394,Fornasetti,unisex,False,10047569,https://cdn-images.farfetch-contents.com/10/04...,https://cdn-images.farfetch-contents.com/10/04...,False,...,SGD,NaN,154.0701,$207,$207,200.0,NaN,False,face print plate,2
188815,175,"[{'scaleId': 0, 'size': 'One Size'}]",59394,Fornasetti,unisex,False,10047567,https://cdn-images.farfetch-contents.com/10/04...,https://cdn-images.farfetch-contents.com/10/04...,False,...,SGD,NaN,154.0701,$207,$207,200.0,NaN,False,whisker print plate,10


In [17]:
farfetchData['priceInfo.finalPrice']=farfetchData['priceInfo.finalPrice'].map(roundPrices)

In [18]:
imagesSource="D:/farfetchdataset/cutout-img/cutout/"

In [19]:
images=pandas.concat([farfetchData['images.cutOut'],farfetchData['priceInfo.finalPrice']],axis=1)

In [20]:
images

,images.cutOut,priceInfo.finalPrice
0,https://cdn-images.farfetch-contents.com/13/41...,800.0
1,https://cdn-images.farfetch-contents.com/13/78...,300.0
2,https://cdn-images.farfetch-contents.com/13/37...,900.0
3,https://cdn-images.farfetch-contents.com/13/74...,200.0
4,https://cdn-images.farfetch-contents.com/13/66...,900.0
...,...,...
188812,https://cdn-images.farfetch-contents.com/10/04...,200.0
188813,https://cdn-images.farfetch-contents.com/10/04...,200.0
188814,https://cdn-images.farfetch-contents.com/10/04...,200.0
188815,https://cdn-images.farfetch-contents.com/10/04...,200.0


In [21]:
sampledImages=images.sample(frac=1.0,random_state=1).reset_index(drop=True)

In [22]:
sampledImages

,images.cutOut,priceInfo.finalPrice
0,https://cdn-images.farfetch-contents.com/12/90...,600.0
1,https://cdn-images.farfetch-contents.com/12/28...,1300.0
2,https://cdn-images.farfetch-contents.com/14/17...,1100.0
3,https://cdn-images.farfetch-contents.com/11/92...,200.0
4,https://cdn-images.farfetch-contents.com/13/42...,500.0
...,...,...
188812,https://cdn-images.farfetch-contents.com/13/84...,70.0
188813,https://cdn-images.farfetch-contents.com/13/61...,200.0
188814,https://cdn-images.farfetch-contents.com/13/97...,200.0
188815,https://cdn-images.farfetch-contents.com/14/22...,300.0


In [23]:
sampledImages['imagePath']=sampledImages['images.cutOut'].map(lambda x: imagesSource+x.split('/')[-1])

In [24]:
sampledImages['imagePath'][0]

'D:/farfetchdataset/cutout-img/cutout/12903116_13445423_300.jpg'

In [25]:
train,test=train_test_split(sampledImages,train_size=.7,shuffle=True,random_state=1)

In [26]:
train

,images.cutOut,priceInfo.finalPrice,imagePath
50748,https://cdn-images.farfetch-contents.com/13/44...,400.0,D:/farfetchdataset/cutout-img/cutout/13441927_...
123131,https://cdn-images.farfetch-contents.com/13/56...,200.0,D:/farfetchdataset/cutout-img/cutout/13565547_...
67896,https://cdn-images.farfetch-contents.com/13/41...,1400.0,D:/farfetchdataset/cutout-img/cutout/13417612_...
91206,https://cdn-images.farfetch-contents.com/13/34...,80.0,D:/farfetchdataset/cutout-img/cutout/13342789_...
69587,https://cdn-images.farfetch-contents.com/14/03...,400.0,D:/farfetchdataset/cutout-img/cutout/14038683_...
...,...,...,...
73349,https://cdn-images.farfetch-contents.com/12/84...,4000.0,D:/farfetchdataset/cutout-img/cutout/12845040_...
109259,https://cdn-images.farfetch-contents.com/14/06...,500.0,D:/farfetchdataset/cutout-img/cutout/14063350_...
50057,https://cdn-images.farfetch-contents.com/13/77...,600.0,D:/farfetchdataset/cutout-img/cutout/13777392_...
5192,https://cdn-images.farfetch-contents.com/14/17...,900.0,D:/farfetchdataset/cutout-img/cutout/14179971_...


In [27]:
trainer=tensorflow.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,validation_split=.2)
tester=tensorflow.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,)

In [28]:
trainImages=trainer.flow_from_dataframe(dataframe=train,
                            x_col='imagePath',
                            y_col='priceInfo.finalPrice',
                            target_size=(120,120),
                            color_mode='rgb',
                           class_mode='raw',
                           batch_size=32,
                           shuffle=True,
                           seed=42,
                           subset='training')

valImages=trainer.flow_from_dataframe(dataframe=train,
                            x_col='imagePath',
                            y_col='priceInfo.finalPrice',
                            target_size=(120,120),
                            color_mode='rgb',
                           class_mode='raw',
                           batch_size=32,
                           shuffle=True,
                           seed=42,
                           subset='validation')

testImages=tester.flow_from_dataframe(dataframe=test,
                            x_col='imagePath',
                            y_col='priceInfo.finalPrice',
                            target_size=(120,120),
                            color_mode='rgb',
                           class_mode='raw',
                           batch_size=32,
                           shuffle=False)

Found 105737 validated image filenames.
Found 26434 validated image filenames.
Found 56646 validated image filenames.


In [29]:
print("Num GPUs Available: ", len(tensorflow.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [ ]:
input=tensorflow.keras.Input(shape=(120,120,3))
cnn=tensorflow.keras.layers.Conv2D(filters=16,kernel_size=(3,3),activation='relu')(input)
cnn=tensorflow.keras.layers.MaxPool2D()(cnn)
cnn=tensorflow.keras.layers.Conv2D(filters=32,kernel_size=(3,3),activation='relu')(cnn)
cnn=tensorflow.keras.layers.MaxPool2D()(cnn)
cnn=tensorflow.keras.layers.GlobalAveragePooling2D()(cnn)
cnn=tensorflow.keras.layers.Dense(64,activation='relu')(cnn)
output=tensorflow.keras.layers.Dense(1,activation='linear')(cnn)
model=tensorflow.keras.Model(inputs=input,outputs=output)

model.compile(optimizer='adam',loss='mse')

history=model.fit(trainImages,validation_data=valImages,
                  epochs=100,
                  callbacks=[tensorflow.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5)])

Epoch 1/100
3305/3305 [==============================] - 4944s 1s/step - loss: 1986005.2500 - val_loss: 2264094.0000
Epoch 2/100
3305/3305 [==============================] - ETA: 0s - loss: 1940647.0000